In [1]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# define the text data
data = ['The system shall generate an interrupt if any error occurs.',
        'The device shall power off if the temperature exceeds the threshold.',
        'The application shall display an error message if the login fails.',
        'The program shall terminate if the input is invalid.']

# create a tokenizer and fit on the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)

# convert the text data into sequences of numerical data
sequences = tokenizer.texts_to_sequences(data)

# create the sliding window data
window_size = 2
X = []
y = []
for seq in sequences:
    for i in range(window_size, len(seq)):
        context = seq[i - window_size:i]
        target = seq[i]
        X.append(context)
        y.append(target)

# pad the sequences to have equal length
max_len = max([len(seq) for seq in X])
X = pad_sequences(X, maxlen=max_len)

# convert the target to a binary class matrix
num_classes = len(tokenizer.word_index) + 1
y = np.eye(num_classes)[y]


ImportError: cannot import name 'pad_sequences' from 'keras.preprocessing.sequence' (c:\Users\WB\anaconda3\lib\site-packages\keras\preprocessing\sequence.py)

In [6]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
import numpy as np

# Prepare the data
text = "The system shall generate an interrupt if any error occurs."
sentences = text.split('.')
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(sentences)
X = []
y = []
for i in range(len(sequences) - 1):
    X.append(sequences[i])
    y.append(sequences[i+1][1:])  # Skip the first word (the period)
max_length = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=max_length, padding='post')
y = pad_sequences(y, maxlen=max_length, padding='post')
y = to_categorical(y, num_classes=vocab_size)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=None, test_size=0.1, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the LSTM model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# Evaluate the LSTM model
_, acc = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy: %.2f%%' % (acc * 100))

# Use the LSTM model to generate the next sentence
context = "The system shall generate an interrupt"
context_seq = tokenizer.texts_to_sequences([context])[0]
context_seq = pad_sequences([context_seq], maxlen=max_length, padding='post')
next_word_probs = model.predict(context_seq)[0][-1]
next_word_index = np.random.choice(len(next_word_probs), p=next_word_probs)
next_word = tokenizer.index_word[next_word_index]
next_sentence = context + ' ' + next_word + '.'
print(next_sentence)


ValueError: With n_samples=1, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.